In [6]:
import pandas as pd
import sqlite3
import numpy as pd

db_path = 'DB/daily_visits.db'
csv_path = 'data/users_web.csv'

In [7]:
# crear db
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [ ]:
# Crear tabla con numero de usuarios por día
cursor.execute('''
    CREATE TABLE IF NOT EXISTS visit_quant (
        date DATE PRIMARY KEY,
        quant INTEGER
    )
''')
conn.commit()

In [12]:
df_csv = pd.read_csv(csv_path, parse_dates=['Date'])
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    851 non-null    object
 1   Users   851 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.4+ KB


In [16]:
df_csv['Date'] = pd.to_datetime(df_csv['Date'], dayfirst=True)
df_csv['Date']

0     2020-01-01
1     2020-01-02
2     2020-01-03
3     2020-01-04
4     2020-01-05
         ...    
846   2022-04-26
847   2022-04-27
848   2022-04-28
849   2022-04-29
850   2022-04-30
Name: Date, Length: 851, dtype: datetime64[ns]

In [8]:
df_sql = pd.read_sql("SELECT * from visit_quant", conn, parse_dates='Date')
df_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    336 non-null    datetime64[ns]
 1   Users   851 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 13.4 KB


In [80]:
# borrar duplicados
df_new = pd.merge(df_sql, df_csv, indicator=True, how='outer')
df_new = df_new[df_new['_merge'] == 'right_only'].drop('_merge', axis=1)
df_new.count()

date     30
quant    30
dtype: int64

In [81]:
df_new.to_sql('visit_quant', conn, if_exists='append', index=False)

30

In [4]:
#ver todo
cursor.execute('''
    SELECT COUNT(*) FROM visit_quant
''')
cursor.fetchall()

[(851,)]

In [5]:
#ver tipo de date
cursor.execute('''
    SELECT typeof(Date) FROM visit_quant
    LIMIT 1;
''')
cursor.fetchall()

[('text',)]

In [6]:
#borrar tabla
cursor.execute('''
    drop table visit_quant
''')
cursor.fetchall()

[]

In [83]:
cursor.close()
conn.close()